# Investigating conversion of Elastic ingest pipelines into Cribl worker mappers

## Install prerequisite packages

In [1]:
!pip install elasticsearch google-generativeai ipywidgets tqdm

## Create a connection to Elasticsearch

In [2]:
from elasticsearch import Elasticsearch
import os

# Password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = os.getenv('ELASTIC_PASSWORD')

# Create the client instance
client = Elasticsearch(
    "https://es01:9200",
    basic_auth=("elastic", ELASTIC_PASSWORD),
    ca_certs="/certs/ca/ca.crt"
)

# Successful response!
body = client.info().body
# {'name': 'instance-0000000000', 'cluster_name': ...}

body

{'name': 'es01',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'yqxqbj8fQ1qwpZVzo2LICQ',
 'version': {'number': '8.15.1',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '253e8544a65ad44581194068936f2a5d57c2c051',
  'build_date': '2024-09-02T22:04:47.310170297Z',
  'build_snapshot': False,
  'lucene_version': '9.11.1',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

## Retrieve and manipulate the ingest pipeline

In [3]:
PIPELINE_NAME = 'logs-crowdstrike.fdr-1.41.0'

pipeline = client.ingest.get_pipeline(id=PIPELINE_NAME).body

In [4]:
import json

processors = pipeline[PIPELINE_NAME]['processors']

processors_json = json.dumps(processors)

## Connect to Gemini API

In [27]:
system_instruction="""
You are an expert in Elasticsearch ingest pipelines. 

You will receive a JSON representation of an ingest pipeline fragment containing processors. 
Your task is to explain this pipeline step-by-step, describing each processor in the order they appear. 

For each processor:
- Explain what is the transformation performed by the processor in simple terms as transformation instructions.
- Only ptovide instructions, don't create headers and footers.
"""

prompt = """
 Describe the processors in the following ingest pipeline fragment:
"""

In [28]:
import google.generativeai as genai
import os

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

In [29]:
import json
from tqdm.notebook import tqdm

ARRAY_LENGTH=25

def split_array(arr, X):
    return [arr[i:i + X] for i in range(0, len(arr), X)]

model_parameters = {
    "system_instruction": system_instruction
}

model = genai.GenerativeModel('gemini-1.5-flash-latest', **model_parameters)

generation_config=genai.types.GenerationConfig(
    temperature=0,
    
)

combined_response = []
processor_chunks =  split_array(processors, ARRAY_LENGTH)
progress_bar = tqdm(total=len(processor_chunks))

for i in range(len(processor_chunks)):
    response = model.generate_content(prompt + json.dumps(processor_chunks[i]), generation_config=generation_config)
    combined_response.append(response.text)
    progress_bar.update(1)

  0%|          | 0/9 [00:00<?, ?it/s]

In [30]:
print("\n".join(combined_response))

- Rename the field "message" to "event.original" if the field "event.original" is null.
- Decode the JSON string in the field "event.original" and store the result in the field "crowdstrike".
- Remove the field "metadata.host.aid" if it exists.
- Remove the field "metadata.user.UserSid_readable" if it exists.
- Rename the field "metadata" to "crowdstrike.info" if it exists. If the rename fails, append an error message to the field "error.message".
- Parse the timestamp in the field "crowdstrike.UTCTimestamp" and store it in the field "event.created" if the field "event.created" is null. Use the formats "UNIX_MS" and "ISO8601".
- Parse the timestamp in the field "crowdstrike.timestamp" and store it in the field "event.created" if the field "event.created" is null. Use the formats "UNIX_MS" and "ISO8601".
- Parse the timestamp in the field "crowdstrike.CreationTimeStamp" and store it in the field "event.created" if the field "event.created" is null. Use the formats "UNIX" and "ISO8601".


In [32]:
print(combined_response[0])

- Rename the field "message" to "event.original" if the field "event.original" is null.
- Decode the JSON string in the field "event.original" and store the result in the field "crowdstrike".
- Remove the field "metadata.host.aid" if it exists.
- Remove the field "metadata.user.UserSid_readable" if it exists.
- Rename the field "metadata" to "crowdstrike.info" if it exists. If the rename fails, append an error message to the field "error.message".
- Parse the timestamp in the field "crowdstrike.UTCTimestamp" and store it in the field "event.created" if the field "event.created" is null. Use the formats "UNIX_MS" and "ISO8601".
- Parse the timestamp in the field "crowdstrike.timestamp" and store it in the field "event.created" if the field "event.created" is null. Use the formats "UNIX_MS" and "ISO8601".
- Parse the timestamp in the field "crowdstrike.CreationTimeStamp" and store it in the field "event.created" if the field "event.created" is null. Use the formats "UNIX" and "ISO8601".


In [31]:
from IPython.display import display, Markdown

display(Markdown("\n".join(combined_response)))

- Rename the field "message" to "event.original" if the field "event.original" is null.
- Decode the JSON string in the field "event.original" and store the result in the field "crowdstrike".
- Remove the field "metadata.host.aid" if it exists.
- Remove the field "metadata.user.UserSid_readable" if it exists.
- Rename the field "metadata" to "crowdstrike.info" if it exists. If the rename fails, append an error message to the field "error.message".
- Parse the timestamp in the field "crowdstrike.UTCTimestamp" and store it in the field "event.created" if the field "event.created" is null. Use the formats "UNIX_MS" and "ISO8601".
- Parse the timestamp in the field "crowdstrike.timestamp" and store it in the field "event.created" if the field "event.created" is null. Use the formats "UNIX_MS" and "ISO8601".
- Parse the timestamp in the field "crowdstrike.CreationTimeStamp" and store it in the field "event.created" if the field "event.created" is null. Use the formats "UNIX" and "ISO8601".
- Parse the timestamp in the field "crowdstrike.Time" and store it in the field "event.created" if the field "event.created" is null. Use the formats "ISO8601" and "UNIX".
- Parse the timestamp in the field "crowdstrike._time" and store it in the field "event.created" if the field "event.created" is null. Use the formats "ISO8601" and "UNIX".
- Copy the value of the field "event.created" to the field "@timestamp" if the field "event.created" is not null.
- Copy the value of the field "_ingest.timestamp" to the field "@timestamp" if the field "@timestamp" is null.
- If the field "crowdstrike.ContextTimeStamp" is not null and not empty, convert the value from Windows NT timestamp format to UNIX.
- If the field "crowdstrike.StartTime" is not null, convert the value from Windows NT timestamp format to UNIX.
- Parse the timestamp in the field "crowdstrike.StartTime" and store it in the field "crowdstrike.StartTime" if the field "crowdstrike.StartTime" is not null. Use the format "UNIX".
- If the field "crowdstrike.EndTime" is not null, convert the value from Windows NT timestamp format to UNIX.
- Parse the timestamp in the field "crowdstrike.EndTime" and store it in the field "crowdstrike.EndTime" if the field "crowdstrike.EndTime" is not null. Use the format "UNIX".
- Rename the field "crowdstrike.message" to "message" if it exists.
- Convert the value of the field "crowdstrike.CurrentLocalIP" to an IP address if the field is not null and not empty.
- Parse the timestamp in the field "crowdstrike.FirstDiscoveredDate" and store it in the field "crowdstrike.FirstDiscoveredDate" if the field is not null and not empty. Use the format "UNIX".
- Convert the value of the field "crowdstrike.aipCount" to an integer if the field is not null and not empty.
- Convert the value of the field "crowdstrike.discovererCount" to an integer if the field is not null and not empty.
- Convert the value of the field "crowdstrike.localipCount" to an integer if the field is not null and not empty.
- Generate a unique ID for the document based on the fields "@timestamp", "crowdstrike.id", "crowdstrike.aid", and "crowdstrike.cid" if they exist.
- Set the value of the field "ecs.version" to "8.11.0".
- **Add fields from a map to the event:**
  - Iterate over the `params` map.
  - For each key-value pair in the map:
    - If the value is a list, add it to the `ctx.event` field as a new list.
    - Otherwise, add it to the `ctx.event` field as a single value.

- **Rename the `crowdstrike.id` field to `event.id`:**
  - Rename the field `crowdstrike.id` to `event.id`.
  - Ignore missing fields.

- **Rename the `crowdstrike.event_simpleName` field to `event.action`:**
  - Rename the field `crowdstrike.event_simpleName` to `event.action`.
  - Ignore missing fields.

- **Convert count fields to long:**
  - Iterate over the `ctx.crowdstrike` map.
  - For each key-value pair in the map:
    - If the key contains "Count" or "Port", try to parse the value as a long.
    - If the parsing fails, do nothing.

- **Remove empty hashes:**
  - Create a new map called `hashes` and a list called `related`.
  - Iterate over the `params` map.
  - For each key-value pair in the map:
    - Remove the key-value pair from `ctx.crowdstrike`.
    - If the value is an empty hash (all 0s), skip to the next key-value pair.
    - Otherwise, add the value to the `hashes` map and the `related.hash` list.
  - Create a new map called `_temp` and add the `hashes` map to it.
  - If the `related.hash` list is not empty, add the `related` list to the `ctx.related` field.

- **Rename the `crowdstrike.aid` field to `observer.serial_number`:**
  - Rename the field `crowdstrike.aid` to `observer.serial_number`.
  - Ignore missing fields and failures.

- **Split the `crowdstrike.aip` field by whitespace:**
  - Split the field `crowdstrike.aip` by whitespace.
  - Ignore missing fields.

- **Convert the `crowdstrike.aip` field to IP address:**
  - Convert the field `crowdstrike.aip` to IP address.
  - Ignore missing fields.

- **Rename the `crowdstrike.aip` field to `observer.ip`:**
  - Rename the field `crowdstrike.aip` to `observer.ip`.
  - Ignore missing fields and failures.

- **Copy the `observer.ip` field to the `observer.address` field:**
  - Copy the value of the `observer.ip` field to the `observer.address` field.
  - Ignore empty values.

- **Rename the `crowdstrike.AgentVersion` field to `observer.version`:**
  - Rename the field `crowdstrike.AgentVersion` to `observer.version`.
  - Ignore missing fields and failures.

- **Rename the `crowdstrike.ConfigBuild` field to `observer.version`:**
  - Rename the field `crowdstrike.ConfigBuild` to `observer.version`.
  - Ignore missing fields and failures.

- **Set the `observer.vendor` field to "crowdstrike":**
  - Set the value of the `observer.vendor` field to "crowdstrike".

- **Set the `observer.type` field to "agent":**
  - Set the value of the `observer.type` field to "agent".

- **Append the `observer.ip` field to the `related.ip` field:**
  - If the `observer.ip` field is not null and is a list, iterate over the list.
  - For each value in the list, append it to the `related.ip` field.
  - Do not allow duplicates.

- **Append the `observer.ip` field to the `related.ip` field:**
  - If the `observer.ip` field is not null and is a list, iterate over the list.
  - For each value in the list, append it to the `related.ip` field.
  - Do not allow duplicates.

- **Rename the `crowdstrike.ComputerName` field to `host.hostname`:**
  - Rename the field `crowdstrike.ComputerName` to `host.hostname`.
  - Ignore missing fields and failures.

- **Copy the `host.hostname` field to the `host.name` field:**
  - Copy the value of the `host.hostname` field to the `host.name` field.
  - Ignore empty values and failures.

- **Append the `crowdstrike.info.host.ComputerName` field to the `related.hosts` field:**
  - If the `crowdstrike.info.host.ComputerName` field is not null, append it to the `related.hosts` field.
  - Do not allow duplicates.

- **Rename the `crowdstrike.info.host.ComputerName` field to `host.name`:**
  - If the `host.name` field is null, rename the field `crowdstrike.info.host.ComputerName` to `host.name`.
  - Ignore missing fields.

- **Append the `host.name` field to the `related.hosts` field:**
  - If the `host.name` field is not null, append it to the `related.hosts` field.
  - Do not allow duplicates.

- **Rename the `crowdstrike.City` field to `host.geo.city_name`:**
  - Rename the field `crowdstrike.City` to `host.geo.city_name`.
  - Ignore missing fields and failures.

- **Rename the `crowdstrike.Continent` field to `host.geo.continent_name`:**
  - Rename the field `crowdstrike.Continent` to `host.geo.continent_name`.
  - Ignore missing fields and failures.

- **Rename the `crowdstrike.Country` field to `host.geo.country_name`:**
  - Rename the field `crowdstrike.Country` to `host.geo.country_name`.
  - Ignore missing fields and failures.

- **Rename the `crowdstrike.Timezone` field to `host.geo.timezone`:**
  - Rename the field `crowdstrike.Timezone` to `host.geo.timezone`.
  - Ignore missing fields and failures.

- Rename the field "crowdstrike.MachineDomain" to "host.domain". If the field is missing, ignore it. If the operation fails, ignore it.
- Convert the field "crowdstrike.info.host.aip" to an IP address and store it in the temporary field "_temp.aip". If the field is missing, ignore it. If the operation fails, ignore it.
- Remove the field "crowdstrike.info.host.aip" if the temporary field "_temp.aip" exists.
- Append the value of the temporary field "_temp.aip" to the field "host.ip" if the temporary field "_temp.aip" exists. Do not allow duplicates.
- Append the value of the temporary field "_temp.aip" to the field "related.ip" if the temporary field "_temp.aip" exists. Do not allow duplicates.
- Set the field "host.os.type" to "linux" if the field "crowdstrike.event_platform" exists and its value is "Lin".
- Set the field "host.os.type" to "macos" if the field "crowdstrike.event_platform" exists and its value is "Mac".
- Set the field "host.os.type" to "windows" if the field "crowdstrike.event_platform" exists and its value is "Win".
- Rename the field "crowdstrike.OSVersionString" to "host.os.version". If the field is missing, ignore it. If the operation fails, ignore it.
- Rename the field "crowdstrike.Version" to "host.os.version". If the field is missing, ignore it. If the operation fails, ignore it.
- Rename the field "crowdstrike.CommandLine" to "process.command_line". If the field is missing, ignore it.
- If the field "process.command_line" exists and is not empty and the field "host.os.type" exists, split the command line string into individual arguments following Windows conventions and store them in the field "process.args". Also, store the number of arguments in the field "process.args_count".
- Rename the field "crowdstrike.ImageFileName" to "process.executable". If the field is missing, ignore it.
- If the field "process.executable" exists and is not empty, extract the process name from the executable path and store it in the field "process.name".
- Convert the field "crowdstrike.ExitCode" to a long integer. If the field is missing, ignore it.
- Rename the field "crowdstrike.ExitCode" to "process.exit_code". If the field is missing, ignore it.
- Convert the field "crowdstrike.ProcessStartTime" to a string. If the field is missing, ignore it.
- Convert the field "crowdstrike.ProcessEndTime" to a string. If the field is missing, ignore it.
- If the fields "crowdstrike.ProcessStartTime" and "crowdstrike.ProcessEndTime" exist and are not empty, calculate the process uptime in seconds and store it in the field "process.uptime".
- If the field "crowdstrike.ProcessStartTime" exists and is not empty, parse it as a UNIX timestamp and store it in the field "crowdstrike.ProcessStartTime".
- Rename the field "crowdstrike.ProcessStartTime" to "process.start". If the field is missing, ignore it.
- If the field "crowdstrike.ProcessEndTime" exists and is not empty, parse it as a UNIX timestamp and store it in the field "crowdstrike.ProcessEndTime".
- Rename the field "crowdstrike.ProcessEndTime" to "process.end". If the field is missing, ignore it.
- Convert the field "crowdstrike.RawProcessId" to a long integer. If the field is missing, ignore it.
- Rename the field "crowdstrike.RawProcessId" to "process.pid". If the field is missing, ignore it. 

- Convert the field "crowdstrike.TargetProcessId" to a string if it is not null and not already a string.
- Rename the field "crowdstrike.TargetProcessId" to "process.entity_id", ignoring missing fields.
- Convert the field "crowdstrike.ParentProcessId" to a string if it is not null and not already a string.
- Rename the field "crowdstrike.ParentProcessId" to "process.parent.entity_id", ignoring missing fields.
- Rename the field "crowdstrike.ParentBaseFileName" to "process.parent.name", ignoring missing fields.
- Convert the field "crowdstrike.ProcessGroupId" to a long, ignoring missing fields.
- Rename the field "crowdstrike.ProcessGroupId" to "process.pgid", ignoring missing fields.
- Rename the field "crowdstrike.ContextProcessId" to "process.entity_id", ignoring missing fields and failures, if the field "process.entity_id" is null.
- Convert the field "crowdstrike.ContextThreadId" to a long, ignoring missing fields, if the field "process.thread.id" is null.
- Rename the field "crowdstrike.ContextThreadId" to "process.thread.id", ignoring missing fields and failures, if the field "process.thread.id" is null.
- Convert the field "crowdstrike.EtwRawProcessId" to a long, ignoring missing fields.
- Rename the field "crowdstrike.EtwRawProcessId" to "process.pid", ignoring missing fields, if the field "process.pid" is null.
- Convert the field "crowdstrike.EtwRawThreadId" to a long, ignoring missing fields.
- Rename the field "crowdstrike.EtwRawThreadId" to "process.thread.id", ignoring missing fields, if the field "process.thread.id" is null.
- Rename the field "crowdstrike.ServiceDisplayName" to "process.title", ignoring missing fields.
- Rename the field "_temp.hashes" to "process.hash", if the field "event.action" is not null and contains "Process" or "Service", and the field "_temp.hashes" is not null and has a size greater than 0.
- Rename the field "crowdstrike.UID" to "user.id", ignoring missing fields.
- Rename the field "crowdstrike.info.user.UserName" to "user.name", ignoring missing fields, if the field "crowdstrike.info.user.UserName" is not null and the field "user.name" is null.
- Rename the field "crowdstrike.info.user.User" to "user.name", ignoring missing fields, if the field "crowdstrike.info.user.User" is not null and the field "user.name" is null.
- Rename the field "crowdstrike.GID" to "user.group.id", ignoring missing fields.
- Rename the field "crowdstrike.UserSid" to "user.id", ignoring missing fields, if the field "user.id" is null or empty.
- Append the value "admin" to the field "user.roles", if the field "crowdstrike.UserIsAdmin" is equal to "1".
- Rename the field "crowdstrike.UserName" to "user.name", ignoring missing fields, if the field "crowdstrike.UserName" is not null and the field "user.name" is null.
- Split the field "crowdstrike.UserPrincipal" into an array "_temp.user_parts" using the separator "@", if the field "crowdstrike.UserPrincipal" is not null.
- Rename the field "crowdstrike.UserPrincipal" to "user.email", ignoring missing fields. 

- Set the `user.domain` field to the second element of the `_temp.user_parts` array if the array exists and has two elements.
- Set the `user.full_name` field to the first element of the `_temp.user_parts` array if the array exists and has two elements.
- Append the value of the `user.name` field to the `related.user` array if the `user.name` field exists.
- Append the value of the `crowdstrike.info.user.User` field to the `related.user` array if the `crowdstrike.info.user.User` field exists.
- Append the value of the `user.full_name` field to the `related.user` array if the `user.full_name` field exists.
- Set the `source.ip` field to the value of the `crowdstrike.CurrentLocalIP` field if the `source.ip` field is null and the `crowdstrike.CurrentLocalIP` field exists.
- Set the `source.ip` field to the value of the `crowdstrike.LocalIP` field if the `source.ip` field is null and the `crowdstrike.LocalIP` field exists.
- Split the `crowdstrike.LocalAddressIP4` field into an array using whitespace as the separator if the `crowdstrike.LocalAddressIP4` field exists.
- Convert the `crowdstrike.LocalAddressIP4` field to an IP address if it is an array and has at least one element.
- Set the `source.ip` field to the first element of the `crowdstrike.LocalAddressIP4` array if the `source.ip` field is null and the `crowdstrike.LocalAddressIP4` field is an array with at least one element.
- Split the `crowdstrike.LocalAddressIP6` field into an array using whitespace as the separator if the `crowdstrike.LocalAddressIP6` field exists.
- Convert the `crowdstrike.LocalAddressIP6` field to an IP address if it is an array and has at least one element.
- Set the `source.ip` field to the first element of the `crowdstrike.LocalAddressIP6` array if the `source.ip` field is null and the `crowdstrike.LocalAddressIP6` field is an array with at least one element.
- Set the `source.address` field to the value of the `source.ip` field if the `source.ip` field is not empty.
- Rename the `crowdstrike.LocalPort` field to `source.port` if the `crowdstrike.LocalPort` field exists.
- Convert the `crowdstrike.RemoteAddressIP4` field to an IP address if the field exists.
- Rename the `crowdstrike.RemoteAddressIP4` field to `destination.ip` if the `crowdstrike.RemoteAddressIP4` field exists.
- Set the `destination.address` field to the value of the `destination.ip` field if the `destination.ip` field is not empty.
- Convert the `crowdstrike.RemoteAddressIP6` field to an IP address if the field exists.
- Rename the `crowdstrike.RemoteAddressIP6` field to `destination.ip` if the `crowdstrike.RemoteAddressIP6` field exists.
- Set the `destination.address` field to the value of the `destination.ip` field if the `destination.ip` field is not empty.
- Rename the `crowdstrike.RemotePort` field to `destination.port` if the `crowdstrike.RemotePort` field exists.
- Rename the `crowdstrike.Protocol` field to `network.iana_number` if the `crowdstrike.Protocol` field exists.
- If the `network.iana_number` field exists, use a script to set the `network.transport` field based on the value of the `network.iana_number` field.
- Set the `network.direction` field to `outbound` if the `crowdstrike.ConnectionDirection` field is equal to `0`.

- Set the `network.direction` field to `inbound` if the `ctx.crowdstrike.ConnectionDirection` field is equal to `1`.
- Set the `network.direction` field to `unknown` if the `ctx.network.direction` field is null and the `ctx.crowdstrike.ConnectionDirection` field is not null and not empty.
- Ignore missing or failed community ID.
- If the `ctx.crowdstrike.LocalAddressIP4` field is a list and has at least one element, append each element to the `related.ip` field.
- If the `ctx.crowdstrike.LocalAddressIP6` field is a list and has at least one element, append each element to the `related.ip` field.
- If the `ctx.source.ip` field is not null and not empty, append it to the `related.ip` field.
- If the `ctx.destination.ip` field is not null and not empty, append it to the `related.ip` field.
- If the `ctx.crowdstrike.LocalAddressIP4` field is a list and has at least one element, append each element to the `related.ip` field.
- If the `ctx.crowdstrike.LocalAddressIP6` field is a list and has at least one element, append each element to the `related.ip` field.
- If the `ctx.source.ip` field is not null and not empty, append it to the `related.ip` field.
- If the `ctx.destination.ip` field is not null and not empty, append it to the `related.ip` field.
- Rename the `crowdstrike.MAC` field to `source.mac`.
- If the `ctx.source.mac` field is null, rename the `crowdstrike.PhysicalAddress` field to `source.mac`.
- Convert the `source.mac` field to uppercase.
- Rename the `crowdstrike.DownloadServer` field to `server.address`.
- Rename the `crowdstrike.DownloadPath` field to `url.path`.
- If the `ctx.url.path` field is not null and does not start with `/`, prepend `/` to the `url.path` field.
- Extract the registered domain from the `server.address` field and store it in the `server` field.
- Set the `url.scheme` field to `https` if the `ctx.crowdstrike.DownloadPort` field is equal to `443`.
- Set the `url.scheme` field to `http` if the `ctx.crowdstrike.DownloadPort` field is not equal to `443`.
- Set the `url.full` field to the concatenation of `url.scheme`, `server.address`, and `url.path` if all three fields are not null.
- Extract the URI parts from the `url.full` field.
- Extract the registered domain from the `url.domain` field and store it in the `url` field.
- Extract the geolocation information from the `observer.ip` field and store it in the `observer.geo` field.
- Extract the geolocation information from the `source.ip` field and store it in the `source.geo` field.

- **geoip**:  Look up the IP address in the `destination.ip` field using the default GeoIP database and store the results in the `destination.geo` field. If the `destination.ip` field is missing, skip this step.
- **geoip**: Look up the IP address in the `source.ip` field using the `GeoLite2-ASN.mmdb` database and store the `asn` and `organization_name` properties in the `source.as` field. Only perform the lookup for the first IP address if multiple are present. If the `source.ip` field is missing, skip this step.
- **geoip**: Look up the IP address in the `destination.ip` field using the `GeoLite2-ASN.mmdb` database and store the `asn` and `organization_name` properties in the `destination.as` field. If the `destination.ip` field is missing, skip this step.
- **rename**: Rename the `source.as.asn` field to `source.as.number`. If the `source.as.asn` field is missing, skip this step.
- **rename**: Rename the `source.as.organization_name` field to `source.as.organization.name`. If the `source.as.organization_name` field is missing, skip this step.
- **rename**: Rename the `destination.as.asn` field to `destination.as.number`. If the `destination.as.asn` field is missing, skip this step.
- **rename**: Rename the `destination.as.organization_name` field to `destination.as.organization.name`. If the `destination.as.organization_name` field is missing, skip this step.
- **set**: Set the `dns.type` field to `query` if the `ctx.event.action` field is equal to `DnsRequest`.
- **registered_domain**: Extract the registered domain from the `crowdstrike.DomainName` field and store it in the `dns.question` field. If the `crowdstrike.DomainName` field is missing, skip this step. This processor is only executed if the `ctx.event.action` field is equal to `DnsRequest`.
- **rename**: Rename the `dns.question.domain` field to `dns.question.name`. This processor is only executed if the `ctx.event.action` field is equal to `DnsRequest`. If the `dns.question.domain` field is missing, skip this step.
- **script**: Map the decimal DNS request type from the `ctx.crowdstrike.RequestType` field to its name and store it in the `dns.question.type` field. This processor is only executed if the `ctx.event.action` field is equal to `DnsRequest` and the `ctx.crowdstrike.RequestType` field is not null or empty.
- **convert**: Convert the value in the `crowdstrike.Size` field to a long integer. If the `crowdstrike.Size` field is missing, skip this step. If the conversion fails, ignore the error.
- **rename**: Rename the `crowdstrike.Size` field to `file.size`. If the `crowdstrike.Size` field is missing, skip this step.
- **rename**: Rename the `crowdstrike.FileIdentifier` field to `file.inode`. If the `crowdstrike.FileIdentifier` field is missing, skip this step.
- **rename**: Rename the `crowdstrike.SourceFileName` field to `file.path`. If the `crowdstrike.SourceFileName` field is missing, skip this step.
- **rename**: Rename the `crowdstrike.TargetFileName` field to `file.path`. If the `crowdstrike.TargetFileName` field is missing, skip this step. If the renaming fails, ignore the error.
- **rename**: Rename the `crowdstrike.DiskParentDeviceInstanceId` field to `file.device`. If the `crowdstrike.DiskParentDeviceInstanceId` field is missing, skip this step.
- **set**: Set the `file.type` field to `file` if the `ctx.file.path` field is not null and the `ctx.event.action` field does not contain `Directory`.
- **set**: Set the `file.type` field to `dir` if the `ctx.file.path` field is not null and the `ctx.event.action` field contains `Directory` or the `ctx.file.path` field ends with `\` or `/`.
- **script**: Extract file information from the `ctx.file.path` field and store it in the `file` field. This processor is only executed if the `ctx.file.path` field is not null and has a length greater than 1.
- **rename**: Rename the `_temp.hashes` field to `file.hash`. This processor is only executed if the `ctx.event.action` field contains `File`, `Directory`, or `Executable` and the `_temp.hashes` field is not null and has a size greater than 0.
- **set**: Copy the value from the `crowdstrike.SensorId` field to the `device.id` field. If the `crowdstrike.SensorId` field is empty, skip this step.
- **set**: Copy the value from the `crowdstrike.DeviceId` field to the `device.id` field if the `device.id` field is null. If the `crowdstrike.DeviceId` field is empty, skip this step.
- **split**: Split the values in the `crowdstrike.FalconGroupingTags` field by the separator `,` and store the resulting values in an array. If the `crowdstrike.FalconGroupingTags` field is missing, skip this step. If the splitting fails, ignore the error.
- **split**: Split the values in the `crowdstrike.SensorGroupingTags` field by the separator `,` and store the resulting values in an array. If the `crowdstrike.SensorGroupingTags` field is missing, skip this step. If the splitting fails, ignore the error. 

- Split the field "crowdstrike.Tags" into multiple fields using "," and optional whitespace as the separator. Ignore missing fields and failures.
- Split the field "crowdstrike.CallStackModuleNames" into multiple fields using "|" as the separator. Ignore missing fields and failures.
- Convert the field "crowdstrike.UserTime" to a long integer. Ignore missing fields.
- Convert the field "crowdstrike.KernelTime" to a long integer. Ignore missing fields.
- Convert the field "crowdstrike.CycleTime" to a long integer. Ignore missing fields.
- Append the value of "crowdstrike.ConfigStateHash" to the field "related.hash" if "crowdstrike.ConfigStateHash" is not null or empty. Ignore failures and do not allow duplicates.
- Trim whitespace from the field "crowdstrike.BootArgs". Ignore missing fields.
- Split the field "crowdstrike.BootArgs" into multiple fields using whitespace as the separator. Ignore missing fields.
- Convert the field "crowdstrike.LogonTime" to a date field using the UNIX format if "crowdstrike.LogonTime" is not null or empty.
- Convert the field "crowdstrike.LogoffTime" to a date field using the UNIX format if "crowdstrike.LogoffTime" is not null or empty.
- Convert the field "crowdstrike.ConnectTime" to a date field using the UNIX format if "crowdstrike.ConnectTime" is not null or empty.
- Convert the field "crowdstrike.PreviousConnectTime" to a date field using the UNIX format if "crowdstrike.PreviousConnectTime" is not null or empty.
- Convert the field "crowdstrike.AgentLocalTime" to a date field using the UNIX format if "crowdstrike.AgentLocalTime" is not null or empty.
- Convert the field "crowdstrike.FirstSeen" to a date field using the UNIX format if "crowdstrike.FirstSeen" is not null or empty.
- Convert the field "crowdstrike.ContextTimeStamp" to a date field using the UNIX format if "crowdstrike.ContextTimeStamp" is not null or empty.
- Convert the field "crowdstrike.BiosReleaseDate" to a date field using the formats "MM/dd/yyyy" and "strict_date_optional_time" if "crowdstrike.BiosReleaseDate" is not null or empty.
- Convert the field "crowdstrike.AgentTimeOffset" to a float. Ignore missing fields.
- Convert the field "crowdstrike.Timeout" to a long integer. Ignore missing fields.
- Convert the field "crowdstrike.PhysicalAddressLength" to a long integer. Ignore missing fields.
- Convert the field "crowdstrike.InterfaceIndex" to a long integer. Ignore missing fields.
- Convert the field "crowdstrike.NetLuidIndex" to a long integer. Ignore missing fields.
- Convert the field "crowdstrike.AttemptNumber" to a long integer. Ignore missing fields.
- Convert the field "crowdstrike.SystemTableIndex" to a long integer. Ignore missing fields.
- Split the field "crowdstrike.NeighborList" into multiple fields using "|" as the separator. Ignore missing fields.
- Split the field "crowdstrike.ConfigStateData" into multiple fields using "|" as the separator. Ignore missing fields. 

- Append the value of the `crowdstrike.LogonServer` field to the `related.hosts` field if the `crowdstrike.LogonServer` field is not null.
- Append the value of the `crowdstrike.ClientComputerName` field to the `related.hosts` field if the `crowdstrike.ClientComputerName` field is not null.
- Append the value of the `crowdstrike.info.user.LastLoggedOnHost` field to the `related.hosts` field if the `crowdstrike.info.user.LastLoggedOnHost` field is not null.
- Remove the `crowdstrike.event_platform` field if the `host.os.type` field is not null.
- Remove the fields `_temp`, `crowdstrike.timestamp`, `crowdstrike._time`, `crowdstrike.Time`, `crowdstrike.CreationTimeStamp`, `crowdstrike.DomainName`, `crowdstrike.ConnectionDirection`, `crowdstrike.UserIsAdmin`, `crowdstrike.UTCTimestamp`, `crowdstrike.TargetDirectoryName`, and `aws`.
- Iterate over the whole document and remove fields with null values, empty strings, hyphens, "none" strings, empty maps, and empty lists.
- Execute the ingest pipeline named `global@custom`.
- Execute the ingest pipeline named `logs@custom`.
- Execute the ingest pipeline named `logs-crowdstrike.integration@custom`.
- Execute the ingest pipeline named `logs-crowdstrike.fdr@custom`. 
